# Taxi-v3 project

## Project Details
Actions: ['n', 'w', 'e', 's', 'p', 'd']
state var: taxi loc, passenger loc

When the episode starts, the taxi starts off at a random square and the passenger is at a random location. 
The taxi drives to the passenger's location, picks up the passenger, drives to the passenger's destination 
(another one of the four specified locations), and then drops off the passenger. Once the passenger is dropped off, 
the episode ends.

    Rewards:
    There is a default per-step reward of -1,
    except for delivering the passenger, which is +20,
    or executing "pickup" and "drop-off" actions illegally, which is -10.

    Passenger locations:
    - 0: R(ed)
    - 1: G(reen)
    - 2: Y(ellow)
    - 3: B(lue)
    - 4: in taxi

    Destinations:
    - 0: R(ed)
    - 1: G(reen)
    - 2: Y(ellow)
    - 3: B(lue)

    Actions:
    There are 6 discrete deterministic actions:
    - 0: move south
    - 1: move north
    - 2: move east
    - 3: move west
    - 4: pickup passenger
    - 5: drop off passenger



## Dependency packages

In [53]:
import numpy as np
import pandas as pd
import random

In [54]:
MAP = [
    "+---------+",
    "|R: | : :G|",
    "| : | : : |",
    "| : : : : |",
    "| | : | : |",
    "|Y| : |B: |",
    "+---------+",
]
n_rows = 5
n_columns = 5
total_dest_loc = 4
total_states = 500
dest_idx_array = {'R': 0, 'G': 1, 'Y': 2, 'B': 3}


In [71]:
class taxi_v3:

    def encode(self, taxi_row, taxi_col, pass_loc, dest_idx):
        # (5) 5, 5, 4
        i = taxi_row
        i *= 5
        i += taxi_col
        i *= 5
        i += pass_loc
        i *= 4
        i += dest_idx
        return i

    def decode(self, i):
        out = []
        out.append(i % 4)
        i = i // 4
        out.append(i % 5)
        i = i // 5
        out.append(i % 5)
        i = i // 5
        out.append(i)
        assert i >= 0 or i < 5
        return reversed(out)

    def _loc_to_dest_idx(self, i, j):
        index = i * n_columns + j
        for loc, dest_idx in self.loc_dest_array.items():
            if (loc == index):
                return dest_idx
        return -1

    def __init__(self):
        self.done = False
        self.actions = ['n', 'w', 'e', 's', 'p', 'd']
        self.table = pd.DataFrame({s: {a : None for a in self.actions} for s in range(total_states)})
        self.terminal_states = {}
        self.loc_dest_array = {}
        self.rewards_tbl = {'default': -1, 'illegal_pick': -10, 'illegal_drop': -10, 'success': +20, 'outside': -40}

    def _get_next_state(self, state, action):
        taxi_row, taxi_col, pass_loc, dest_idx = self.decode(state)
        done = False
        if (action == 'p'):
            if (pass_loc != 4 and  self._loc_to_dest_idx(taxi_row, taxi_col) == pass_loc):
                pass_loc = 4
        elif (action == 'd'):
            if (pass_loc == 4 and self._loc_to_dest_idx(taxi_row, taxi_col) == dest_idx):
                done = True
                return state, done
        else:
            movement = {'s': (+1, 0), 'n': (-1, 0), 'e': (0, +1), 'w': (0, -1)}
            if (taxi_row * n_columns + taxi_col, action) in self.terminal_states.items():
                done = True
                return state, done
            
            row_delta, col_delta = movement[action]
            taxi_row, taxi_col = taxi_row + row_delta, taxi_col + col_delta
            if (taxi_row < 0 or taxi_row >= n_rows):
                done = True
                return state, done
            if (taxi_col < 0 or taxi_col >= n_columns):
                done = True
                return state, done
      
        next_state = self.encode(taxi_row, taxi_col, pass_loc, dest_idx)
        return (next_state, done)


    def step(self, action):
        if self.done:
            assert "done is True. Step cannot be used"
        
        next_state, done = self._get_next_state(self.state, action)
        reward = self._get_rewards_for_transistion(self.state, action, next_state)
        
        self.done = done
        self.state = next_state
        return next_state, reward, done

    def serialize(self, Map):
        i = 0
        j = 0
        size = len(Map)
        for map_row in range(1, size - 1):
            row_str = Map[map_row]
            str_size = len(row_str)
            j = 0
            for idx in range(1, str_size - 1):
                if (row_str[idx] == ':'):
                    continue
                elif (row_str[idx] == '|'):
                    self.terminal_states[i * n_columns + j - 1] = 'e'
                    self.terminal_states[i * n_columns + j] = 'w'
                elif (row_str[idx] == ' '):
                    j = j + 1
                else:
                    print(i,j)
                    self.loc_dest_array[i * n_columns + j] = dest_idx_array[row_str[idx]]
                    j = j + 1
            i = i + 1

    def set_rewards(self, rewards):
        for keys,val in rewards:
            self.rewards_tbl[keys] = val

    def _get_rewards_for_transistion(self, state, action, next_state):
         taxi_row, taxi_col, pass_loc, dest_idx = self.decode(state)
         if (action == 'p'):
            #print('pass_loc: {} current_loc: {}'.format(pass_loc, self._loc_to_dest_idx(taxi_row, taxi_col)))
            if (pass_loc != 4 and self._loc_to_dest_idx(taxi_row, taxi_col) == pass_loc):
                return self.rewards_tbl['default']
            else:
                return self.rewards_tbl['illegal_pick']
         elif (action == 'd'):
            if (pass_loc == 4 and self._loc_to_dest_idx(taxi_row, taxi_col) == dest_idx):
                return self.rewards_tbl['success']
            else:
                return self.rewards_tbl['illegal_drop']
         else:
             if (state == next_state):
                 return self.rewards_tbl['outside']
             else:
                return self.rewards_tbl['default']
       
    def reset(self):
        self.done = False       
        pass_loc = random.randint(0, total_dest_loc - 1)
        dest_idx = random.randint(0, total_dest_loc - 1)
        taxi_row = random.randint(0, n_rows - 1)
        taxi_col = random.randint(0, n_columns - 1)

        self.state = self.encode(taxi_row, taxi_col, pass_loc, dest_idx)
        return self.state
    
    def render(self):
        taxi_row, taxi_col, pass_loc, dest_idx = self.decode(self.state)

    def run_episode(self, action_list):
        self.reset()

        for action in action_list:
            if self.done:
                break
            next_state, reward, done = self.step(action)
            state = next_state
    
    def print_fn(self, state):
        taxi_row, taxi_col, pass_loc, dest_idx = self.decode(state)
        dest_idx_row = 0
        dest_idx_col = 0
        for key,value in self.loc_dest_array.items():
            if (value == dest_idx):
                dest_idx_row = key // n_columns
                dest_idx_col = key  % n_columns
                break
        
        pass_loc_row = 0
        pass_loc_col = 0
        if pass_loc != 4:
            for key,value in self.loc_dest_array.items():
                if (value == pass_loc):
                    pass_loc_row = key // n_columns
                    pass_loc_col = key % n_columns
                    break
        else:
            pass_loc_row = taxi_row
            pass_loc_col = taxi_col
        print("taxi_loc: ({},{}) pass_loc: ({},{}) dest_idx:({}, {})".format(taxi_row, taxi_col, pass_loc_row, pass_loc_col, dest_idx_row, dest_idx_col))


In [72]:
taxi = taxi_v3()
taxi.serialize(MAP)
print(taxi.loc_dest_array)

0 0
0 4
4 0
4 3
{0: 0, 4: 1, 20: 2, 23: 3}


In [73]:
for key,value in taxi.terminal_states.items():
    i = key//n_columns
    j = key%n_columns
    print(i,j,value)

for str in MAP:
    print(str)

taxi.reset()
print(*taxi.decode(taxi.state))


0 1 e
0 2 w
1 1 e
1 2 w
3 0 e
3 1 w
3 2 e
3 3 w
4 0 e
4 1 w
4 2 e
4 3 w
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
2 4 2 1


In [74]:
next_state, reward, done = taxi.step('s')
print(*taxi.decode(next_state))


3 4 2 1


In [75]:
action_list = ['s', 'e', 'w', 'e', 's']
taxi.run_episode(action_list)

In [76]:
#SARSA 
ESTIMATED_RETURNS_TBL = pd.DataFrame.from_dict({s : {a: 0 for a in taxi.actions} for s in range(total_states)}, orient='index')
epsilon = 1
min_epsilon = 0.1
n_episodes = 7000
epsilon_decay = 0.999
alpha = 0.1
gamma = 0.9

for _ in range(n_episodes):
    s0 = taxi.reset()

    estimated_returns = ESTIMATED_RETURNS_TBL
    epsilon = max(epsilon, min_epsilon)
    action_probs = np.asarray([epsilon/len(taxi.actions)] * len(taxi.actions), dtype=np.float)
    greedy_action_idx = np.argmax(estimated_returns.loc[s0].values)
    action_probs[greedy_action_idx] += (1 - epsilon)
    a0 = np.random.choice(estimated_returns.columns, p=action_probs)
    done = False

    while not done:
        s1, reward, done = taxi.step(a0)
        action_probs = np.asarray([epsilon/len(taxi.actions)] * len(taxi.actions), dtype=float)
        greedy_action_idx = np.argmax(estimated_returns.loc[s1].values)
        action_probs[greedy_action_idx] += (1 - epsilon)
        a1 = np.random.choice(estimated_returns.columns, p=action_probs)

        ESTIMATED_RETURNS_TBL.loc[s0, a0] += alpha *(reward - ESTIMATED_RETURNS_TBL.loc[s0, a0] + gamma * ESTIMATED_RETURNS_TBL.loc[s1, a1])
        s0, a0 = s1, a1

    epsilon = epsilon * epsilon_decay


In [77]:
print(ESTIMATED_RETURNS_TBL[100: 200])

              n          w          e          s          p          d
100   87.138743 -12.993648  -2.560788  -2.746460  -4.110631  -3.820798
101   -4.772861 -27.520045   4.225246  -4.587018  -6.818161  -5.181064
102   39.919459 -11.957632  -1.395254   2.145898  -3.280610   7.328422
103   13.097634 -22.579701  -2.971116  -0.574683  -3.361025  -1.920953
104   -7.225044 -11.520383  -7.031660  -6.824117  -9.530649  -8.230299
..          ...        ...        ...        ...        ...        ...
195   -3.231512  -3.207618 -26.368920  36.239950  -5.426645  -7.191364
196   -0.566573   1.940070 -12.512062  33.097720  -1.404752   2.305188
197  109.735807  24.365036  47.271442  51.590858  61.645514  57.894336
198    1.757685   1.247614 -17.225710  -2.467735  -3.609919  -2.592829
199    7.188877  22.242811 -14.429943  -0.518699  -2.729886  -2.800000

[100 rows x 6 columns]


In [82]:
print(MAP)
state = taxi.reset()
taxi.print_fn(state)
done = False
action_list = []
while not done:
    action = ESTIMATED_RETURNS_TBL.loc[state].idxmax()

    print("action performed: {}".format(action))
    next_state, reward, done = taxi.step(action)
    taxi.print_fn(next_state)
    state = next_state



['+---------+', '|R: | : :G|', '| : | : : |', '| : : : : |', '| | : | : |', '|Y| : |B: |', '+---------+']
taxi_loc: (2,1) pass_loc: (0,0) dest_idx:(4, 3)
action performed: w
taxi_loc: (2,0) pass_loc: (0,0) dest_idx:(4, 3)
action performed: n
taxi_loc: (1,0) pass_loc: (0,0) dest_idx:(4, 3)
action performed: n
taxi_loc: (0,0) pass_loc: (0,0) dest_idx:(4, 3)
action performed: p
taxi_loc: (0,0) pass_loc: (0,0) dest_idx:(4, 3)
action performed: s
taxi_loc: (1,0) pass_loc: (1,0) dest_idx:(4, 3)
action performed: e
taxi_loc: (1,1) pass_loc: (1,1) dest_idx:(4, 3)
action performed: s
taxi_loc: (2,1) pass_loc: (2,1) dest_idx:(4, 3)
action performed: e
taxi_loc: (2,2) pass_loc: (2,2) dest_idx:(4, 3)
action performed: e
taxi_loc: (2,3) pass_loc: (2,3) dest_idx:(4, 3)
action performed: s
taxi_loc: (3,3) pass_loc: (3,3) dest_idx:(4, 3)
action performed: s
taxi_loc: (4,3) pass_loc: (4,3) dest_idx:(4, 3)
action performed: d
taxi_loc: (4,3) pass_loc: (4,3) dest_idx:(4, 3)


In [68]:
print(taxi.terminal_states)
print((16, 'w') in taxi.terminal_states.items())

{1: 'e', 2: 'w', 6: 'e', 7: 'w', 15: 'e', 16: 'w', 17: 'e', 18: 'w', 20: 'e', 21: 'w', 22: 'e', 23: 'w'}
True


In [1]:
import gym

In [2]:
taxi = gym.make('Taxi-v3')